In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import sophius.utils as utils
import sophius.dataload as dload
from sophius.modelgen import ConvModelGenerator
from sophius.train import train_express_gpu
import torchvision.datasets as dset
import torchvision.transforms as T

VAL_SIZE = 1024

cifar10 = dset.CIFAR10('../data/CIFAR10', train=True, download=True,
                           transform=T.ToTensor())
cifar_gpu = dload.cifar_to_gpu(cifar10)

Files already downloaded and verified


torch.Size([3, 32, 32])

In [3]:
model_gen = ConvModelGenerator((3, 32, 32), 10, conv_num=6, lin_num=3)
model_tmpl = model_gen.generate_model_tmpl()
print(model_tmpl)
fixed_model_gpu = model_tmpl.instantiate_model().type(torch.cuda.FloatTensor)

Conv2d       (128, 10, 10)  (5, 5)   (3, 3)  
ReLU         (128, 10, 10) 
MaxPool2d    (128, 6, 6)    (2, 2)   (2, 2)  
AvgPool2d    (128, 7, 7)    (4, 4)   (1, 1)  
Conv2d       (192, 2, 2)    (4, 4)   (2, 2)  
GlobalAvgPool2d (192, 1, 1)   
Flatten      192           
Linear       128           
Dropout      128            (0.35)  
PReLU        128           
Linear       10            



In [15]:
from fvcore.nn import FlopCountAnalysis

flops = FlopCountAnalysis(fixed_model_gpu, cifar_gpu[0][:1])
flops.total()

Unsupported operator aten::max_pool2d encountered 1 time(s)
Unsupported operator aten::avg_pool2d encountered 2 time(s)
Unsupported operator aten::prelu encountered 1 time(s)


2558720

In [16]:
import pandas as pd
import seaborn as sns

loader_gpu = dload.get_loader_gpu(cifar_gpu, val_size=VAL_SIZE, batch_size=1024)

df = pd.DataFrame(columns=['flops', 'time', 'val_acc', 'train_acc'])

for i in range(10):
    model = model_gen.generate_model().type(torch.cuda.FloatTensor)    
    flops = FlopCountAnalysis(model, cifar_gpu[0][:1]).total()
    # express train
    t, val_acc, train_acc = train_express_gpu(model = model,
                                              train = True,
                                              loader = loader_gpu,
                                              milestones = [],
                                              num_epoch = 1,
                                              verbose = True)
    
    df.loc[i] = [flops, t, val_acc, train_acc]
    print(df.loc[i])

Unsupported operator aten::leaky_relu encountered 1 time(s)
Unsupported operator aten::max_pool2d encountered 1 time(s)


KeyboardInterrupt: 

In [ ]:
sns.lineplot(df, x='time', y='flops')